In [5]:
%run imports

In [6]:
df_valid = pd.read_csv("output/valid_photos.csv")
df_valid.head()

,photo_id,business_id,caption,label
0,zsvj7vloL4L5jhYyPIuVwg,Nk-SJhPlDBkAZvfsADtccA,Nice rock artwork everywhere and craploads of ...,inside
1,HCUdRJHHm_e0OCTlZetGLg,yVZtL5MmrpiivyCIrVkGgA,NaN,outside
2,vkr8T0scuJmGVvN2HJelEA,_ab50qdWOk0DdB6XOrBitw,oyster shooter,drink
3,pve7D6NUrafHW3EAORubyw,SZU9c8V2GuREDN5KgyHFJw,Shrimp scampi,food
4,H52Er-uBg6rNrHcReWTD2w,Gzur0f0XMkrVxIwYJvOt2g,NaN,food


In [8]:
dimension = len(df_valid) // 10
df_reducido, _ = train_test_split(df_valid, train_size=dimension, stratify=df_valid["label"], random_state=12)

df_reducido.head()
print(f"El tamaño del nuevo DataFrame es: {len(df_reducido)}")

El tamaño del nuevo DataFrame es: 19000


In [9]:
df_reducido.to_csv("prueba.csv", index=False)

In [10]:
df_reducido['label'].value_counts()

label
food       10267
inside      5325
outside     1765
drink       1484
menu         159
Name: count, dtype: int64

Este código realiza los siguientes pasos:

1. Cuenta el número de imágenes en cada categoría.
2. Define la secuencia de augmentación utilizando imgaug.
3. Para cada categoría menos "food", calcula cuántas imágenes faltan para alcanzar el número objetivo (target_count).
4. Realiza augmentación de las imágenes en esas categorías y guarda las nuevas imágenes con nombres únicos.
5. Crea un nuevo dataframe con las imágenes aumentadas y combina este dataframe con el dataframe original.
6. Verifica el balanceo de categorías.

In [2]:
import imgaug.augmenters as iaa
import os
import imageio.v3 as imageio
import pandas as pd 


# Definir una secuencia de augmentación
seq = iaa.Sequential([
    iaa.Fliplr(0.5),   # Voltear horizontalmente el 50% de las imágenes
    iaa.Crop(percent=(0, 0.1)),   # Recortar entre 0% a 10% de cada lado de la imagen
    iaa.GaussianBlur(sigma=(0, 3.0))   # Aplicar desenfoque gaussiano con sigma entre 0 y 3.0
])

In [11]:
# Contar el número de imágenes por categoría
category_counts = df_reducido['label'].value_counts()

# El número objetivo de imágenes por categoría (cota superior de homogeinizacion)
target_count = int(5000)
# Ruta donde están almacenadas las imágenes
image_dir = 'photos'

# Lista para almacenar las imágenes aumentadas
augmented_images = []

# Aumentar imágenes de cada categoría excepto 'food'
for category, count in tqdm(category_counts.items(), desc='Processing Categories'):
    if count < target_count:
        images_to_augment = df_reducido[df_reducido['label'] == category]
        num_augmentations = target_count - count
        
        for index, row in images_to_augment.iterrows():
            image_path = os.path.join(image_dir, f"{row['photo_id']}.jpg")
            image = imageio.imread(image_path)
            
            for i in range(num_augmentations // len(images_to_augment) + 1):
                image_aug = seq(image=image)
                augmented_image_path = os.path.join(image_dir, f"aug_{row['photo_id'][0]}_{i}.jpg")
                imageio.imwrite(augmented_image_path, image_aug)
                
                augmented_images.append({'photo_id': f"aug_{row['photo_id'][0]}_{i}", 'label': category})

# Crear un nuevo dataframe con las imágenes aumentadas
df_augmented = pd.DataFrame(augmented_images)

# Combinar el dataframe original con el dataframe de imágenes aumentadas
df_balanced = pd.concat([df_reducido, df_augmented], ignore_index=True)

# Verificar el balanceo de categorías
print(df_balanced['label'].value_counts())

Processing Categories: 5it [01:49, 21.95s/it]

label
food       10267
drink       5936
inside      5325
outside     5295
menu        5088
Name: count, dtype: int64
